# Lecture 16

## Systems of Differential Equations III:

### Phase Planes and Stability

In [ ]:
import numpy as np
import sympy as sp
import scipy.integrate
sp.init_printing()
##################################################
##### Matplotlib boilerplate for consistency #####
##################################################
from ipywidgets import interact
from ipywidgets import FloatSlider
from matplotlib import pyplot as plt

import cmath

%matplotlib inline

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg')

global_fig_width = 8
global_fig_height = global_fig_width / 1.61803399
font_size = 12

plt.rcParams['axes.axisbelow'] = True
plt.rcParams['axes.edgecolor'] = '0.8'
plt.rcParams['axes.grid'] = True
plt.rcParams['axes.labelpad'] = 8
plt.rcParams['axes.linewidth'] = 2
plt.rcParams['axes.titlepad'] = 16.0
plt.rcParams['axes.titlesize'] = font_size * 1.4
plt.rcParams['figure.figsize'] = (global_fig_width, global_fig_height)
plt.rcParams['font.sans-serif'] = ['Computer Modern Sans Serif', 'DejaVu Sans', 'sans-serif']
plt.rcParams['font.size'] = font_size
plt.rcParams['grid.color'] = '0.8'
plt.rcParams['grid.linestyle'] = 'dashed'
plt.rcParams['grid.linewidth'] = 2
plt.rcParams['lines.dash_capstyle'] = 'round'
plt.rcParams['lines.dashed_pattern'] = [1, 4]
plt.rcParams['xtick.labelsize'] = font_size
plt.rcParams['xtick.major.pad'] = 4
plt.rcParams['xtick.major.size'] = 0
plt.rcParams['ytick.labelsize'] = font_size
plt.rcParams['ytick.major.pad'] = 4
plt.rcParams['ytick.major.size'] = 0
##################################################

## Recap from previous parts

- 2-D linear systems can be solved analytically
- Eigenvalues are important
- Some larger systems can be simplified
- Phase planes, nullclines and fixed points

## Example from previous lecture


\begin{eqnarray*}
\dot{x} &=& x(1-x) -xy,\\
\dot{y} &=& y\left(2-y\right) - 3xy.
\end{eqnarray*}

In [ ]:
def dX_dt(X, t):
    return np.array([ X[0]*(1. - X[0]) - X[0]*X[1],
                     2.*X[1]*(1.-X[1]/2.) -3*X[0]*X[1]])

def plot_phase_plane():
    
    plt.figure(figsize=(10,10))
    
    init_x = [1.05, 0.9, 0.7, 0.5, 0.5, 0.32, 0.1]
    init_y = [1.0, 1.3, 1.6, 1.8, 0.2, 0.2, 0.2]
    
    plt.plot(init_x, init_y, 'g*', markersize=20)
    
    for v in zip(init_x,init_y):
        X0 = v                              # starting point
        X = scipy.integrate.odeint( dX_dt, X0, np.linspace(0,10,100))         # we don't need infodict here
        plt.plot( X[:,0], X[:,1], lw=3, color='green')
    
    
    
    # plot nullclines
    x = np.linspace(-0.1,1.1,24)
    y = np.linspace(-0.1,2.1,24)
    
    plt.hlines(0,-1,15, color='#F39200', lw=4, label='y-nullcline 1')
    plt.plot(x,1 - x, color='#0072bd', lw=4, label='x-nullcline 2')
    plt.vlines(0,-1,15, color='#0072bd', lw=4, label='x-nullcline 1')
    plt.plot(x,2 - 3*x, color='#F39200', lw=4, label='y-nullcline 2')

    # quiverplot - define a grid and compute direction at each point
    X , Y  = np.meshgrid(x, y)                  # create a grid
    DX = X*(1-X) - X*Y                          # evaluate dx/dt
    DY = 2*Y*(1 - Y/2.0) - 3*X*Y                # evaluate dy/dt               
    M = (np.hypot(DX, DY))                      # norm growth rate 
    M[ M == 0] = 1.                             # avoid zero division errors 

    plt.quiver(X, Y, DX/M, DY/M, M)
    plt.xlim(-0.05,1.1)
    plt.ylim(-0.05,2.1)
    plt.xlabel('x')
    plt.ylabel('y')

In [ ]:
plot_phase_plane()

## Linear ODEs for understanding nonlinear
The decoupled ODE system 


\begin{eqnarray*}
\dot{x} = \frac{\rm{d}x}{\rm{d}t} &=& \lambda_1 x,\\
\dot{y} = \frac{\rm{d}y}{\rm{d}t} &=& \lambda_2 y,
\end{eqnarray*}


has a **fixed point** or **steady state** where $\;\dot{x}=\dot{y}=0\;$, at the origin.

Solutions look like $\;x=Ae^{\lambda_1 t}\;$ and $\;y=Be^{\lambda_2 t}\;$ and thus grow exponentially
or shrink exponentially depending on the values of $\;\lambda_1\;$ and $\;\lambda_2.$

If $\;\lambda_1 < 0\;$ and $\;\lambda_2 < 0\;$ then all the flow is towards the
fixed point.  If $\;\lambda_1\;$ or $\;\lambda_2\;$ is positive then some flow will be driven away (towards infinity).

Adding in a constant (inhomogeneous) component shifts the fixed point
away from the origin.  Where is the fixed point of 


\begin{eqnarray*}
\dot{x}  &=& \lambda_1 x + 10,\\
\dot{y}  &=& \lambda_2 y + 10?
\end{eqnarray*}

Coupling the system has the effect of altering the principle directions over
which the exponential terms apply (**changing the basis**).

This means that the **homogeneous** linear system


\begin{eqnarray*}
\dot{x}  &=& a x +  b y,\\
\dot{y}  &=& c x +  d y,
\end{eqnarray*}


has a fixed point at the origin.  The long-term growth or shrinkage of solutions over time is determined by the eigenvalues of the matrix

$$
A = \left(
\begin{array}{cc} a & b \\ c& d \end{array}
\right).
$$

More generally, the **inhomogeneous** linear system


\begin{eqnarray*}
\dot{x}  &=& a x +  b y + p,\\
\dot{y}  &=& c x +  d y + q,
\end{eqnarray*}


can be written

$$
\left(
\begin{array}{c} \dot{x} \\ \dot{y} \end{array}
\right)
=
\left(
\begin{array}{cc} a & b \\ c& d \end{array}
\right)
\left(
\begin{array}{c} x \\ y \end{array}
\right)
+
\left(
\begin{array}{c} p \\ q \end{array}
\right)
.
$$

It has a fixed point at

$$
\left(
\begin{array}{c} x \\ y \end{array}
\right)
=
-\left(
\begin{array}{cc} a & b \\ c& d \end{array}
\right)^{-1}
\left(
\begin{array}{c} p \\ q \end{array}
\right)
.
$$

The long-term growth or shrinkage of
solutions over time is again determined by the eigenvalues of the matrix.

## General nonlinear system steady states

A more general two-dimensional nonlinear system is

\begin{eqnarray*}
\dot{x} &=& f(x,y),\\
\dot{y} &=& g(x,y),
\end{eqnarray*}

where $\;f\;$ and $\;g\;$ can be any functions whatever.

We can write a polynomial (Taylor) expansion for the system when it is close to a fixed point $\;(x^*, y^*)\;$ for which $\;f(x^*,y^*)=g(x^*,y^*)=0.$

\begin{eqnarray*}
\dot{x} &=& f(x^*,y^*) + \frac{\partial f}{\partial x}(x-x^*) + \frac{\partial f}{\partial y}(y-y^*) + \ldots,\\
\dot{y} &=& g(x^*,y^*) + \frac{\partial g}{\partial x}(x-x^*) + \frac{\partial g}{\partial y}(y-y^*) + \ldots,
\end{eqnarray*}


So, close to the fixed point:

$$
\left(
\begin{array}{c} \dot{x} \\ \dot{y} \end{array}
\right)
\approx
\left(
\begin{array}{cc} \frac{\partial f}{\partial x} & \frac{\partial f}{\partial y} \\\frac{\partial g}{\partial x} & \frac{\partial g}{\partial y} \end{array}
\right)
\left(
\begin{array}{c} x-x^* \\ y-y^* \end{array}
\right)
.
$$

This means that (really close to the fixed point) we can approximate
with a linear system.   The eigenvalues $\;\lambda_1,\;\lambda_2\;$ of the matrix

$$
J = \left(
\begin{array}{cc} \frac{\partial f}{\partial x} & \frac{\partial f}{\partial y} \\\frac{\partial g}{\partial x} & \frac{\partial g}{\partial y} \end{array}
\right)
$$

will determine if a small perturbation away from  $\;(x^*,\;y^*)\;$ will
decay or grow.

## Steady state classification
$$
J = \left(
\begin{array}{cc} \frac{\partial f}{\partial x} & \frac{\partial f}{\partial y} \\\frac{\partial g}{\partial x} & \frac{\partial g}{\partial y} \end{array}
\right)
$$


- $\lambda_1<\lambda_2<0$ Stable node
- $\lambda_1=\lambda_2<0$ Stable star
- $\lambda_1>\lambda_2>0$ Unstable node
- $\lambda_1=\lambda_2>0$ Unstable star
- $\lambda_1<0<\lambda_2$ Saddle (or hyperbolic) point:  unstable
- Complex $\lambda$: Spiral (with real part determining stability)
- Imaginary $\lambda$: Neutral (solution cycles round fixed point)


The presence of negative eigenvalues determines whether a steady state is physically viable.

### Eigenvalues of $J$
$$
|J-\lambda I| = \left|
\begin{array}{cc} \frac{\partial f}{\partial x}-\lambda & \frac{\partial f}{\partial y} \\\frac{\partial g}{\partial x} & \frac{\partial g}{\partial y}-\lambda \end{array}\right|
= \left(\frac{\partial f}{\partial x}-\lambda\right)\left(\frac{\partial g}{\partial y}-\lambda\right)-\frac{\partial f}{\partial y}\frac{\partial g}{\partial x}
$$

So eigenvalues $\;\lambda\;$ satisfy

$$
\lambda^2 - \lambda\left(\frac{\partial f}{\partial x} + \frac{\partial g}{\partial y}\right) + \frac{\partial f}{\partial x}\frac{\partial g}{\partial y} - \frac{\partial f}{\partial y}\frac{\partial g}{\partial x},
$$

or

$$
\lambda^2 - \lambda\tau + \Delta \quad \rm{where} \quad \tau=\rm{Trace}(J) \quad and \quad \Delta = \rm{Det}(J)
$$

![](assets/phase_diagram.svg)

## Example system

\begin{align*}
\dot{x} &= x(1-x) -xy &=f(x)\\
\dot{y} &= y\left(2-y\right) - 3xy &=g(x).
\end{align*}


- What are the nullclines?
- What are the fixed points?
- What is the stability of the fixed points?

### Calculate the Jacobian

$$
J = \left(
\begin{array}{cc} \frac{\partial f}{\partial x} & \frac{\partial f}{\partial y} \\\frac{\partial g}{\partial x} & \frac{\partial g}{\partial y} \end{array}
\right) = \left(\begin{array}{cc} -2x-y+1 & -x \\-3y & -3x-2y+2 \end{array}\right)
$$

At $(0,0)$:

$$
J_{(0,0)} = \left(\begin{array}{cc} 1 & 0 \\0 & 2 \end{array}\right)
$$

$$
\left|\begin{array}{cc} 1-\lambda & 0 \\0 & 2-\lambda\end{array}\right|\quad\implies\quad\lambda_1 = 1, \; \lambda_2 = 2 \qquad\rm{(unstable)}
$$

### Calculate the Jacobian

$$
J = \left(
\begin{array}{cc} \frac{\partial f}{\partial x} & \frac{\partial f}{\partial y} \\\frac{\partial g}{\partial x} & \frac{\partial g}{\partial y} \end{array}
\right) = \left(\begin{array}{cc} -2x-y+1 & -x \\-3y & -3x-2y+2 \end{array}\right)
$$

At $(1/2,1/2)$:

$$
J_{(1/2,1/2)} = \left(\begin{array}{cc} -\frac{1}{2} & -\frac{1}{2} \\-\frac{3}{2} & -\frac{1}{2} \end{array}\right)
$$

$$
\left|\begin{array}{cc} -\frac{1}{2}-\lambda & -\frac{1}{2} \\-\frac{3}{2} & -\frac{1}{2}-\lambda\end{array}\right|\quad\implies\quad\lambda_\pm = \frac{1}{2} \pm \frac{\sqrt{3}}{2} \qquad\rm{(saddle)}
$$

### Calculate the Jacobian

$$
J = \left(
\begin{array}{cc} \frac{\partial f}{\partial x} & \frac{\partial f}{\partial y} \\\frac{\partial g}{\partial x} & \frac{\partial g}{\partial y} \end{array}
\right) = \left(\begin{array}{cc} -2x-y+1 & -x \\-3y & -3x-2y+2 \end{array}\right)
$$

At $(1,0)$:

$$
J_{(1,0)} = \left(\begin{array}{cc} -1 & -1 \\ 0 & -1 \end{array}\right)
$$

$$
\left|\begin{array}{cc} -1-\lambda & -1 \\0 & -1-\lambda\end{array}\right|\quad\implies\quad\lambda_1 = \lambda_2 = -1 \qquad\rm{(stable)}
$$

### Calculate the Jacobian

$$
J = \left(
\begin{array}{cc} \frac{\partial f}{\partial x} & \frac{\partial f}{\partial y} \\\frac{\partial g}{\partial x} & \frac{\partial g}{\partial y} \end{array}
\right) = \left(\begin{array}{cc} -2x-y+1 & -x \\-3y & -3x-2y+2 \end{array}\right)
$$

At $(0,2)$:

$$
J_{(0,2)} = \left(\begin{array}{cc} -1 & 0 \\ -6 & -2 \end{array}\right)
$$

$$
\left|\begin{array}{cc} -1-\lambda & 0 \\-6 & -2-\lambda\end{array}\right|\quad\implies\quad\lambda_1 = -1, \; \lambda_2 = -2 \qquad\rm{(stable)}
$$

In [ ]:
plot_phase_plane()

## Summary

- Eigenvalues for behaviour of  linear systems
- Eigenvalues for the stability of nonlinear systems